<a href="https://colab.research.google.com/github/Imadamrii/Sentiment-_Analysis_Stocks/blob/main/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk pandas
!pip install torch transformers


In [30]:
import requests
import urllib.request
import time
from datetime import date
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np


In [31]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
api_key = 'LE53BVH2ZNDQJJI8'
# Function to get news sentiment data
def get_news_sentiment(ticker):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&limit=1&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    return data

get_news_sentiment("AAPL")

In [33]:
api_key= 'O7UZDLZIQ490OIJ3'

In [35]:
#loading finbert model
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [62]:
from datetime import datetime, time
import pytz
def is_market_hours(publish_time_utc):
    # Convert publish time to Eastern Time
    eastern = pytz.timezone('US/Eastern')
    publish_time_et = publish_time_utc.astimezone(eastern)

    # Define market hours (9:30 AM to 4:00 PM ET)
    market_start = time(9, 30)
    market_end = time(16, 0)

    # Check if publish time is during market hours
    return int(market_start <= publish_time_et.time() <= market_end)

In [63]:
def analyze_sentiment(article):
    sia = SentimentIntensityAnalyzer()
    sentiment_score = sia.polarity_scores(article)
    if sentiment_score['compound'] >= 0.05:
        return 'Positive', sentiment_score['compound']
    elif sentiment_score['compound'] <= -0.05:
        return 'Negative', sentiment_score['compound']
    else:
        return 'Neutral', sentiment_score['compound']


In [64]:
def analyze_finbert_sentiment(text):
    result = finbert(text)[0]
    return result['label'], result['score']




In [ ]:
import os
import json
import requests
import pandas as pd
from datetime import datetime, timedelta

cache_dir = 'time_series_cache'

def get_daily_time_series(symbol, api_key):
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    cache_file = os.path.join(cache_dir, f'{symbol}.json')

    if os.path.exists(cache_file):
        print(f"Loading cached data for {symbol}")
        with open(cache_file, 'r') as f:
            data = json.load(f)
    else:
        print(f"Fetching data for {symbol} from API")
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
        response = requests.get(url)
        data = response.json()
        if 'Time Series (Daily)' in data:
            with open(cache_file, 'w') as f:
                json.dump(data, f)
        else:
            print(f"Failed to fetch daily time series for {symbol}. Response data: {data}")
            return pd.DataFrame()

    df = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index', dtype=float)
    df.index = pd.to_datetime(df.index)
    return df


symbols = ['AAPL', 'MSFT']

for symbol in symbols:
    df = get_daily_time_series(symbol, api_key)
    print(f"\n{symbol} Data:")
    print(df.head())


In [65]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_market_news(tickers, api_key, days=7):
    # Calculate the date range
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=days)

    # Format dates in YYYYMMDDTHHMM format
    time_to = end_date.strftime('%Y%m%dT%H%M')
    time_from = start_date.strftime('%Y%m%dT%H%M')

    # Initialize an empty DataFrame
    df = pd.DataFrame(columns=['ticker', 'title', 'sentiment', 'summary', 'datetime','Market hour', 'url'])

    for ticker in tickers:
        url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}'
        response = requests.get(url)
        data = response.json()

        # Check if 'feed' key exists in the response data
        if 'feed' not in data:
            print(f"No 'feed' key found in response for ticker {ticker}. Response data: {data}")
            continue

        articles_data = []
        for article in data['feed']:
            # Convert time_published to datetime object
            publish_time_utc = datetime.strptime(article['time_published'], '%Y%m%dT%H%M%S')
            # Check if article was published during market hours
            market_hour = is_market_hours(publish_time_utc)
            # Append data to list
            articles_data.append({
                'ticker': ticker,
                'title': article['title'],
                'sentiment': article['overall_sentiment_label'],
                'summary': article['summary'],
                'datetime': publish_time_utc,
                'Market hourt': market_hour,

                'url': article['url']
            })

        # Convert list of dicts to DataFrame and concatenate
        articles_df = pd.DataFrame(articles_data)
        df = pd.concat([df, articles_df], ignore_index=True)

    return df

In [66]:
def get_market_news_sentiment(tickers,  days, api_key):
  data= get_market_news(tickers, days, api_key )
  data['article'] = data.apply(lambda row: row['title'] + ' ' + row['summary'], axis=1)
  data['nltk_sentiment'], data['nltk_sentiment_score'] = zip(*data['article'].apply(analyze_sentiment))
  # data['finbert_sentiment'], data['finbert_score'] = zip(*data['article'].apply(analyze_finbert_sentiment))
  return data



In [67]:
tickers = ['AAPL', 'MSFT']
days = 7
df = get_market_news_sentiment(tickers, api_key, days)



In [68]:
df
Total buys Shell for a bargain of 2 million

,ticker,title,sentiment,summary,datetime,Market hour,url,Market hourt,article,nltk_sentiment,nltk_sentiment_score
0,AAPL,Is Affirm Holdings Stock a Buy?,Somewhat-Bullish,The fintech is up more than 75% this year and ...,2023-10-28 09:15:00,NaN,https://www.fool.com/investing/2023/10/28/is-a...,0.0,Is Affirm Holdings Stock a Buy? The fintech is...,Positive,0.1531
1,AAPL,Google Spent $26.3B In 2021 To Keep Default Se...,Neutral,"In 2021, Alphabet Inc.'s GOOG GOOGL Google, le...",2023-10-28 06:25:15,NaN,https://www.benzinga.com/news/23/10/35477483/s...,0.0,Google Spent $26.3B In 2021 To Keep Default Se...,Neutral,0.0000
2,AAPL,Add Buffett's Berkshire Hathaway To 'Magnifice...,Somewhat-Bullish,"The 'Magnificent Seven,' a group comprising se...",2023-10-28 03:16:25,NaN,https://www.benzinga.com/news/23/10/35477096/i...,0.0,Add Buffett's Berkshire Hathaway To 'Magnifice...,Positive,0.9552
3,AAPL,Apple's Vision Pro AR Headset Can Diagnose Men...,Somewhat-Bullish,Apple Inc. AAPL is planning to use its first-g...,2023-10-28 03:10:00,NaN,https://www.benzinga.com/news/23/10/35477084/a...,0.0,Apple's Vision Pro AR Headset Can Diagnose Men...,Positive,0.7717
4,AAPL,Artificial intelligence (AI) market 2023-2027-...,Somewhat-Bullish,Artificial intelligence ( AI ) market 2023-202...,2023-10-28 02:30:00,NaN,https://www.prnewswire.com/news-releases/artif...,0.0,Artificial intelligence (AI) market 2023-2027-...,Positive,0.7351
...,...,...,...,...,...,...,...,...,...,...,...
764,MSFT,"3 Stocks That Turned $1,000 Into $1.1 Million ...",Somewhat-Bullish,"Yes, 1,100-fold returns are possible in consum...",2023-10-21 14:00:00,NaN,https://www.fool.com/investing/2023/10/21/3-st...,1.0,"3 Stocks That Turned $1,000 Into $1.1 Million ...",Positive,0.4576
765,MSFT,Around 76 locations were raided in India as pa...,Neutral,76 locations raided in crackdown on tech suppo...,2023-10-21 13:20:00,NaN,https://www.businessinsider.com/scam-call-cent...,0.0,Around 76 locations were raided in India as pa...,Negative,-0.4588
766,MSFT,What Is Shorting a Stock?,Somewhat-Bullish,"You might have heard traders refer to ""shortin...",2023-10-21 13:00:16,NaN,https://www.kiplinger.com/investing/stocks/wha...,0.0,What Is Shorting a Stock? You might have heard...,Positive,0.4871
767,MSFT,Future Tech Titans: 2 Top AI Stocks to Watch i...,Bullish,These two technology stocks can emerge as soli...,2023-10-21 12:30:00,NaN,https://www.fool.com/investing/2023/10/21/futu...,0.0,Future Tech Titans: 2 Top AI Stocks to Watch i...,Positive,0.6705


In [57]:
df[df['ticker']=='AAPL'].date.info

<bound method Series.info of 0      2023-10-28
1      2023-10-28
2      2023-10-28
3      2023-10-28
4      2023-10-28
          ...    
187    2023-10-22
188    2023-10-22
189    2023-10-22
190    2023-10-22
191    2023-10-21
Name: date, Length: 192, dtype: object>

In [16]:
print(df.title.iloc[0])
sentiment_distribution = df['sentiment'].value_counts(normalize=True) * 100
print(sentiment_distribution)
print(df[['sentiment','nltk_sentiment']])
df.to_csv('output.csv', index=False, encoding='utf-8-sig', sep=';')

When Will the Activists Feast?
Somewhat-Bullish    55.0
Neutral             39.0
Bullish              5.0
Somewhat-Bearish     1.0
Name: sentiment, dtype: float64
           sentiment                                     nltk_sentiment
0            Neutral  (Positive, {'neg': 0.0, 'neu': 0.899, 'pos': 0...
1   Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.852, 'pos': 0...
2            Neutral  (Negative, {'neg': 0.138, 'neu': 0.813, 'pos':...
3            Neutral  (Positive, {'neg': 0.0, 'neu': 0.832, 'pos': 0...
4            Neutral  (Positive, {'neg': 0.0, 'neu': 0.938, 'pos': 0...
..               ...                                                ...
95  Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.63, 'pos': 0....
96  Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.96, 'pos': 0....
97  Somewhat-Bullish  (Positive, {'neg': 0.0, 'neu': 0.797, 'pos': 0...
98           Neutral  (Positive, {'neg': 0.053, 'neu': 0.811, 'pos':...
99  Somewhat-Bullish                         

In [ ]:
class sent_analysis:
  def _init_(self,stock_ticker):
